In [1]:
using InteractiveUtils, DrWatson, Comonicon
if isdefined(Main, :IJulia) && Main.IJulia.inited
    using Revise
else
    ENV["GKSwstype"] = 100 # suppress warnings during gif saving
end
versioninfo()
@quickactivate

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU E5-2620 v3 @ 2.40GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 10


In [2]:
using Plots, ProgressMeter, Logging
theme(:bright; size=(300, 300))

In [3]:
using Random, Turing, BayesianSymbolic
using ExprOptimization.ExprRules

includef(args...) = isdefined(Main, :Revise) ? includet(args...) : include(args...)
includef(srcdir("utility.jl"))
includef(srcdir("app_inf.jl"))
includef(srcdir("sym_reg.jl"))
includef(srcdir("exp_max.jl"))
includef(srcdir("analyse.jl"))
includef(srcdir("dataset.jl"))
# Suppress warnings of using _varinfo
with_logger(SimpleLogger(stderr, Logging.Error)) do
    includef(srcdir("scenarios", "magnet.jl"))
    includef(srcdir("scenarios", "nbody.jl"))
    includef(srcdir("scenarios", "bounce.jl"))
    includef(srcdir("scenarios", "mat.jl"))
end

┌ Info: Precompiling BayesianSymbolic [d95aa7d0-ea3e-4103-b443-9ed45b862455]
└ @ Base loading.jl:1317
┌ Info: Skipping precompilation since __precompile__(false). Importing BayesianSymbolic [d95aa7d0-ea3e-4103-b443-9ed45b862455].
└ @ Base loading.jl:1025
┌ Warning: skipping callee #_#38 (called by nothing) due to UndefRefError()
└ @ LoweredCodeUtils /afs/inf.ed.ac.uk/user/s16/s1672897/.julia/packages/LoweredCodeUtils/poBmh/src/signatures.jl:279


In [4]:
using PyCall
pd = pyimport("pandas")

PyObject <module 'pandas' from '/afs/inf.ed.ac.uk/user/s16/s1672897/miniconda3/envs/ml/lib/python3.9/site-packages/pandas/__init__.py'>

In [68]:
function report_synth(dataset; rg_seed=1:5, rg_shuffleseed=1:5, namewithweak=false)
    dataset = "synth/$dataset"
    df = pd.DataFrame(
        columns=("Method", "Normliased RMSE (train)", "Normliased RMSE (test)", "EM seed", "Data seed")
    )
    forces = Dict{Int, Any}()

    count = 1
    @showprogress for seed in rg_seed, shuffleseed in rg_shuffleseed
        hps = (ntrains=5, seed=seed, shuffleseed=shuffleseed)
        if namewithweak
            hps = (hps..., weak=false)
        end

        dir = datadir(dataset)
        scenarios, attributes = loaddata(dir, hps.ntrains; shuffleseed=hps.shuffleseed)
        scenarios_test, attributes_test = loaddata(dir, 20; shuffleseed=hps.shuffleseed, istrain=false)

        local res
        try
            res = wload(resultsdir(dataset, savename(hps; connector="-"), "em.jld2"))
        catch
            println("seed $(hps.seed) unfinished")
        end

        @unpack ScenarioModel, latentname, ealg, malg, elike, mlike, trace = res

        for (force, method) in [(trace[end].force, "BSP"), (ZeroForce(), "Zero")]
            if method == "BSP"
                forceexpr = BayesianSymbolic.get_executable(force.tree, force.grammar)
                @info "Count $count" forceexpr
                # Track expressions
                forces[count] = (force.constant, forceexpr)
            end
            evalres = evalforce(ScenarioModel, scenarios, attributes, force, ealg, elike, mlike)
            evalres_test = evalforce(ScenarioModel, scenarios_test, attributes_test, force, ealg, elike, mlike)

            # Add to data frame
            df.loc[count] = [method, evalres.normrmse, evalres_test.normrmse, seed, shuffleseed]
            count = count + 1
        end
    end
    
    return df, forces
end

function savedf(df, dataset)
    df.groupby(["Method"]).get_group("BSP").to_csv("pretty_paper/raw/em-$dataset-bsp.csv", columns=["Normliased RMSE (test)", "EM seed", "Data seed"], header=false)
    df.groupby(["Method"]).get_group("Zero").to_csv("pretty_paper/raw/em-$dataset-zero.csv", columns=["Normliased RMSE (test)", "EM seed", "Data seed"], header=false)
end

savedf (generic function with 1 method)

NBODY

In [51]:
df, forces = report_synth("nbody")
df

┌ Info: 
│   size(attributes[1]) = (3,)
│   size(ms[1]) = (3,)
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/bayesian_symbolic_physics/src/analyse.jl:79
┌ Info: 
│   size(attributes[1]) = (3,)
│   size(ms[1]) = (3,)
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/bayesian_symbolic_physics/src/analyse.jl:79
┌ Info: 
│   size(attributes[1]) = (3,)
│   size(ms[1]) = (3,)
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/bayesian_symbolic_physics/src/analyse.jl:79
┌ Info: 
│   size(attributes[1]) = (3,)
│   size(ms[1]) = (3,)
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/bayesian_symbolic_physics/src/analyse.jl:79
┌ Info: 
│   size(attributes[1]) = (3,)
│   size(ms[1]) = (3,)
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/bayesian_symbolic_physics/src/analyse.jl:79
┌ Info: 
│   size(attributes[1]) = (3,)
│   size(ms[1]) = (3,)
└ @ Main /afs/inf.ed.ac.uk/user/s16/s1672897/projects/bayesian_symbolic_physics/src/analyse.jl:79
┌ Info: 
│   size(attributes[1]) =

,Method,Normliased RMSE (train),Normliased RMSE (test),EM seed,Data seed
0,BSP,0.684291,0.789963,1,1
1,Zero,1.318852,1.767886,1,1
2,BSP,0.601253,1.067499,1,2
3,Zero,1.301966,1.767886,1,2
4,BSP,0.583036,0.783484,1,3
5,Zero,1.477320,1.767886,1,3
6,BSP,0.659762,0.785545,1,4
7,Zero,1.705750,1.767886,1,4
8,BSP,0.380639,0.771298,1,5
9,Zero,1.077788,1.767886,1,5


In [ ]:
savedf(df, "nbody")

In [59]:
df.groupby(["Method"]).median()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.610192,0.805368
Zero,1.318852,1.767886


In [19]:
df.groupby(["Method"]).quantile([.25, .75])

PyObject              Normliased RMSE (train)  Normliased RMSE (test)
Method                                                      
BSP    0.25                 0.583066                0.783145
       0.75                 0.729911                1.071264
Zero   0.25                 1.301966                1.767886
       0.75                 1.477320                1.767886

BOUNCE

In [70]:
df, forces = report_synth("bounce")
df

┌ Info: Count 1
│   forceexpr = c2 * bmul(bmul(mi, bdiv(bsub(pi, c), bnorm(bsub(pi, c)))), does_collide(si, pi, sj, pj))
└ @ Main In[68]:31
┌ Info: Count 3
│   forceexpr = c2 * bmul(bmul(mi, bdiv(bsub(pi, c), bnorm(bsub(pi, c)))), does_collide(si, pi, sj, pj))
└ @ Main In[68]:31
┌ Info: Count 5
│   forceexpr = c1 * bmul(bmul(bdiv(bmul(mi, mj), mj), bdiv(bsub(pi, c), bnorm(bsub(pi, c)))), does_collide(si, pi, sj, pj))
└ @ Main In[68]:31
┌ Info: Count 7
│   forceexpr = c3 * bmul(bmul(bpow2(bnorm(bsub(vi, vj))), bdiv(bsub(pi, c), bnorm(bsub(pi, c)))), does_collide(si, pi, sj, pj))
└ @ Main In[68]:31
┌ Info: Count 9
│   forceexpr = c2 * bmul(bmul(bdiv(bpow2(bproject(bsub(vi, vj), bnormalize(bsub(pi, c)))), bnorm(bsub(pi, c))), bnormalize(bsub(pi, c))), does_collide(si, pi, sj, pj))
└ @ Main In[68]:31
Progress:  20%|████████▎                                |  ETA: 0:13:36┌ Info: Count 11
│   forceexpr = c3 * bmul(bmul(bdiv(bpow2(mi), mi), bnormalize(bsub(pi, pj))), does_collide(si, pi, sj, 

,Method,Normliased RMSE (train),Normliased RMSE (test),EM seed,Data seed
0,BSP,0.014592,0.031284,1,1
1,Zero,0.040964,0.059844,1,1
2,BSP,0.014345,0.031053,1,2
3,Zero,0.046966,0.059844,1,2
4,BSP,0.024974,0.030933,1,3
5,Zero,0.060070,0.059844,1,3
6,BSP,0.060910,0.021666,1,4
7,Zero,0.117996,0.059844,1,4
8,BSP,0.060931,0.032982,1,5
9,Zero,0.119737,0.059844,1,5


In [26]:
df.groupby(["Method"]).median()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.024974,0.031563
Zero,0.060070,0.059844


In [27]:
df.groupby(["Method"]).quantile([.25, .75])

PyObject              Normliased RMSE (train)  Normliased RMSE (test)
Method                                                      
BSP    0.25                 0.014592                0.031053
       0.75                 0.055263                0.039364
Zero   0.25                 0.046966                0.059844
       0.75                 0.117996                0.059844

MAT

In [28]:
df, forces = report_synth("mat"; rg_seed=0:9, rg_shuffleseed=-1:1, namewithweak=true)
df

Progress: 100%|█████████████████████████████████████████| Time: 0:04:14


,Method,Normliased RMSE (train),Normliased RMSE (test),EM seed,Data seed
0,BSP,0.000235,0.000759,0,-1
1,Zero,0.001148,0.001199,0,-1
2,BSP,0.000212,0.000358,0,0
3,Zero,0.000972,0.001199,0,0
4,BSP,0.000028,0.000539,0,1
5,Zero,0.000579,0.001199,0,1
6,BSP,0.000304,0.001642,1,-1
7,Zero,0.001148,0.001199,1,-1
8,BSP,0.000212,0.000358,1,0
9,Zero,0.000972,0.001199,1,0


In [29]:
df.groupby(["Method"]).median()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.000212,0.000539
Zero,0.000972,0.001199


In [30]:
df.groupby(["Method"]).quantile([.25, .75])

PyObject              Normliased RMSE (train)  Normliased RMSE (test)
Method                                                      
BSP    0.25                 0.000069                0.000358
       0.75                 0.000212                0.000763
Zero   0.25                 0.000579                0.001199
       0.75                 0.001148                0.001199

In [10]:
df.groupby(["Method"]).mean()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.000169,0.000823
Zero,0.000899,0.001199


In [11]:
df.groupby(["Method"]).std()

,Normliased RMSE (train),Normliased RMSE (test)
Method,,
BSP,0.000112,0.000864
Zero,0.000242,0.000000


In [12]:
df.groupby(["Method"]).min()

,Normliased RMSE (train),Normliased RMSE (test),EM seed,Data seed
Method,,,,
BSP,0.000002,0.000002,0,-1
Zero,0.000579,0.001199,0,-1


In [13]:
df.groupby(["Method"]).max()

,Normliased RMSE (train),Normliased RMSE (test),EM seed,Data seed
Method,,,,
BSP,0.000433,0.004248,9,1
Zero,0.001148,0.001199,9,1
